In [2]:
# Binning if needed in other classifiers
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/graph_train.csv')
y = pd.cut(df['edge_cross_norm'], bins=[-1,-1e-10,1]).to_numpy()
df = df.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0'],axis=1)
list_columns = list(df)

for col in list_columns:
    print(col)
    if col!= 'edge_cross_norm' and col!= 'is_bridge':
        df[col] = pd.qcut(df[col],q=5)

X = df.to_numpy()
df.head()


ModuleNotFoundError: No module named 'sklearn'

In [25]:
# XGBoost data preparation 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df2 = pd.read_csv('../data/graph_train.csv')
df2['is_bridge'] = df2['is_bridge'].astype(float)
yn = (df2['edge_cross_norm']>0).to_numpy(dtype=int)
df2 = df2.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0'],axis=1)
Xn = df2.to_numpy(dtype=np.float64)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn,yn, shuffle=True)

from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(Xn_train,yn_train,verbose = True)
print(xgb)
y_pred = xgb.predict(Xn_test)
pos = np.sum([1 if y == 1 else 0 for y in yn_test])
print('Amount of better or neutral:',pos)
print('Amount of worse:',len(yn_test)-pos)
print('Accuracy in test:',(np.sum([1 if yn_test[i]==y_pred[i] else 0 for i in range(len(y_pred))]))/len(yn_test))